In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# set the path to the folder containing the images
images_folder = "/Users/srushtisatardekar/Downloads/SIGN LANG SRUSH 2/train"

# set the desired size for the images
desired_size = (224, 224)

# read the images and labels
images = []
labels = []
for letter in range(ord('A'), ord('Z')+1):
    folder = os.path.join(images_folder, chr(letter))
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_COLOR)
            img = cv2.resize(img, desired_size)
            images.append(img)
            labels.append(letter - ord('A'))

images = np.stack(images)
labels = np.array(labels)

# split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# normalize the pixel values of the images
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=x_train[0].shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(26, activation='softmax'))

# compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.3f}")
print(f"Test Accuracy: {accuracy:.3f}")

# save the model
model.save("sign_language_classifier.h5")

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# load the trained model
model = load_model("sign_language_classifier.h5")

# set the desired size for the images
desired_size = (224, 224)

# create a dictionary to map class indices to labels
label_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 
              10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S',
              19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}

# start capturing video from the default camera
cap = cv2.VideoCapture(0)

while True:
    # read a frame from the video feed
    ret, frame = cap.read()
    
    # convert the frame to color
    color = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # resize the frame to the desired size
    resized = cv2.resize(color, desired_size)
    
    # normalize the pixel values of the image
    normalized = resized.astype('float32') / 255
    
    # reshape the image to match the input shape of the model
    reshaped = np.reshape(normalized, (1, desired_size[0], desired_size[1], 3))
    
    # use the model to make a prediction
    prediction = model.predict(reshaped)
    
    # get the class label for the prediction
    predicted_class = label_dict[int(np.argmax(prediction))]
    
    # display the class label on the frame
    cv2.putText(frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # display the frame
    cv2.imshow('frame', frame)
    
    # wait for 'q' key to be pressed to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the capture and close all windows
cap.release()
cv2.destroyAllWindows()